In [1]:
import pandas as pd
df=pd.read_csv("train.csv")
d_r=[]
keyword="bbc"
tweets=df["tweets"]
for tweet in tweets:
    if keyword in tweet:
        d_r.append(tweet) #finding the initial pseudorelevant documents

In [2]:
#vocabulary of the corpus
Vocabulary=[]
for d in tweets:
    x=d.split()
    for word in x:
        Vocabulary.append(word)
Vocabulary=list(set(Vocabulary))

In [3]:
#zero the variables
mr,nr,m,n={},{},{},{}
for word in Vocabulary:
    mr[word],nr[word],m[word],n[word]=0,0,0,0

In [4]:
for d in tweets:
    x=d.split()
    y=list(set(x))
    if d in d_r:
        for word in y:
            nd=x.count(word)
            mr[word]+=1 # number of relevant documents containg word
            nr[word]+=nd # total number of documents containing word
    for word in y:
        nd=x.count(word)
        m[word]+=1 # frequency of word in relevant documents
        n[word]+=nd #total frequency of word

In [5]:
#finding documents other than pseudorelevant documents
d_ir=[]
for d in tweets:
    if d in d_r:
        continue
    d_ir.append(d)

In [6]:
weight=[]
if "bbc" in Vocabulary:
    Vocabulary.remove("bbc")
for v in Vocabulary:
    if v in mr.keys():
        w=(mr[v]/m[v])*(nr[v]/n[v])
        w/=400
    else:
        w=0
    weight.append(w)

In [7]:
#finding the directional words
directional_words=[]
from numpy import random
directional_prob=random.multinomial(n=15,pvals=weight)
for i,j in enumerate(directional_prob):
    if j>0:
        directional_words.append(Vocabulary[i])
        #print(Vocabulary[i])
if 'bbc' in directional_words:
    directional_words.remove('bbc')

In [8]:
#construction of new collection of documents using directional words
d_new=[]
for word in directional_words:
    for d in tweets:
        if word in d and (d not in d_r):
            d_new.append(d)
d_new=list(set(d_new))
len(d_new)

107

In [9]:
#calculation of Bayesian Factor
import csv
B21={}
df_rc=len(d_r)
d=str(d_r).split()
tf_rc=len(d)
df_irc=len(d_ir)
d=str(d_ir).split()
tf_irc=len(d)
for row in d_new:
    B21[str(row)]=0
    words=row.split()
    for word in words:
        tf_r,df_r,tf_ir,df_ir=0,0,0,0
        for doc in d_r:
            tf_r+=doc.count(word)
            if word in doc:
                df_r+=1
        for doc in d_ir:
            tf_ir+=doc.count(word)
            if word in doc:
                df_ir+=1
    if tf_ir==0 or df_ir==0:
        B21[str(row)]=0
    else:
        B21[str(row)]+=((tf_r*df_r)/(tf_rc*df_rc))*((tf_irc*df_irc)/(tf_ir*df_ir))

In [10]:
#Reward Transformation
tau=8
d_newr=[] # discarding irrelevant documents from the new collection of documents
for tweet in d_new:
    if B21[str(tweet)]>tau:
        d_newr.append(tweet)
d_r.extend(d_newr)

In [11]:
#update the variables (i.e) state transition
for d in tweets:
    x=d.split()
    y=list(set(x))
    if d in d_r:
        for word in y:
            nd=x.count(word)
            mr[word]+=1 # number of relevant documents containg word
            nr[word]+=nd

In [12]:
import re
import string
import spacy
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.base import TransformerMixin
from gsdmm import MovieGroupProcessArray

In [13]:
np.random.seed(1)
docs = [d for d in d_r]
stopwords = ['this', 'is', 'a', 'the', 'of', 'an', 'that', 'or']
docs_toks = [doc.lower().replace(',', '').replace('.', '').split() for doc in docs]
docs_toks = [[w for w in doc if w not in stopwords] for doc in docs_toks]
mgp_ar = MovieGroupProcessArray(K=15, alpha=0.1, beta=0.1, n_iters=100)
y = mgp_ar.fit(docs_toks)

In [14]:
for i,j in y.items():
    z="".join(i.strip().replace(" ","").replace(","," ").replace("[","").replace("]","").replace("'",""))
    print(z,"--> CLuster:",j)

bbc news bbc world service cuts be outlined staff --> CLuster: 13
bbc world service cuts outlined staff --> CLuster: 7
bbc world service cut lrb rrb quarter its staff losing millions funding foreign office --> CLuster: 7
lrb bbc news rrb major cuts bbc world service bbc world service close five its language services th --> CLuster: 2
major cuts bbc world service bbc world service close five its language services likely --> CLuster: 11
irish times bbc world service confirms cuts bbc world service shed around jobs more --> CLuster: 5
quarter bbc world service staff go foreign office grant reduction --> CLuster: 12
statement bbc world service ahead staff briefings fur details weds bbcworldservice bbccuts --> CLuster: 8
anyway twitter wild andy gray quarter bbc world service staff gets laid and nobody notices --> CLuster: 7
bbc world service plans job cuts lrb rrb bbc said wednesday it plans cut jobs tha --> CLuster: 8
bbc caribbean be shut stabroek news guyana lrb bbc rrb bbc world servic

yes rrb bbc news come fly me show gets second series bbc one --> CLuster: 13
bbc iplayer app ipad tablet iphone gets closer launch dedicated app streams bbc ipla --> CLuster: 12
del chl tube who from bbc series realized delia derbyshire bbc radiophonic workshop --> CLuster: 5
bbc faces criticism weak response insulting comments top gear presenters bbc been in --> CLuster: 11
bbc great british railway journeys series dereham cromer lrb pdtv mvgroup rrb show name bbc great britis --> CLuster: 9
reino unido bbc quer sediar eurovision cardiff acordo com fontes internas bbc reino unido est --> CLuster: 11
serious incidents nhs revealed investigation bbc look east reveals series serious unto bbc essex --> CLuster: 10
bbc iplayer app landing uk ipads thursday in june bbc interactive operations engineer briti --> CLuster: 4
bbc bbc radio programs choir top ten choirs aled jones introduces choirs recently picked choir --> CLuster: 7
interesting bbc iplayer app coming ipad thursday radio delayed

In [15]:
#array version skips topics where 0 docs clustered
pprint(mgp_ar.top_words())

{0: ' bbc world news rrb lrb',
 1: ' bbc world news service lrb',
 2: ' bbc news world service lrb',
 3: ' bbc news world service moscow',
 4: ' bbc world savews news cut',
 5: ' bbc news world cuts service',
 6: ' bbc news world rrb lrb',
 7: ' bbc world news service lrb',
 8: ' bbc rrb world news lrb',
 9: ' bbc news cuts online rrb',
 10: ' bbc news world lrb jokes',
 11: ' bbc world news service good',
 12: ' bbc news world cut new',
 13: ' bbc news world cuts service',
 14: ' bbc news world cut rrb'}


In [16]:
#save to a folder which model creates
mgp_ar.save('example_model')
#load model from folder
mgp_ar_loaded=MovieGroupProcessArray.load('example_model')

Folder example_model already exists, not overwriting it. Exiting!


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'example_model'

In [ ]:
print(mgp_ar_loaded.number_docs)

In [ ]:
print(mgp_ar_loaded.vocab_size)

In [ ]:
print(mgp_ar_loaded.cluster_doc_count)

In [ ]:
print(mgp_ar_loaded.cluster_word_count)

In [ ]:
len(d_newr)

In [ ]:
len(tweets)

In [ ]:
for tweet in d_new:
    print(tweet)

In [ ]:
for tweet in d_r:
    print(tweet)

In [ ]:
for tweet in d_r:
    print(tweet)